# Understanding Object Detection tf.Examlpe w/ TF 1.14 
# !! NOT TF 2.0 !!
### Don't Load TF 2.0 Beta
The tensorflow/models/research/object_detection scripts are NOT 2.0 compatible.   And, you don't want to change them.   You'll get an error on a GFile call in a proto buf utililty.   If you have loaded 2.0, you might have to just restart SageMaker.   I had problems going from 2.0 -> 1.14

First, understand this is the world's worst named class - not an example - it is an Example of data (I guess).   Here I'm using TF 2.0beta - because the Understanding Images required it.    The on-line  tutorial shows you can use tf 1.14

https://www.tensorflow.org/beta/tutorials/load_data/images  
https://www.tensorflow.org/tutorials/load_data/tf_records

code references:
https://github.com/tensorflow/models/blob/master/research/object_detection/utils/dataset_util.py
https://github.com/tensorflow/models/blob/master/research/object_detection/data_decoders/tf_example_decoder.py

-- highly deprecated --
https://stackoverflow.com/questions/46687348/decoding-tfrecord-with-tfslim

You will need these skills!  What is an image, tf.Example, serialized example etc.  
You won't get far with served models without this understanding.

## TensorFlow 1.14

### Do this SECOND, first, do UnderstandingImages.ipynb
But, Understanding Images relied on 2.0.  So this might drive you crazy.  
You want to use 2.0 as soon as possible - but here we use some utilities in the models repo and they have not been upgraded (which is odd - 20190814)



In [1]:
import os, sys, pathlib
import random
import IPython.display as display

import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
from code.models.research.object_detection.data_decoders.tf_example_decoder import TfExampleDecoder

ModuleNotFoundError: No module named 'object_detection'

In [ ]:
# This is needed since we cloned tensorflow/models under code.
cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

# cfa_utils
# DEL? from code.cfa_utils.gen_imagesets import gen_imageset_list
from code.cfa_utils.example_utils import voc_to_tfrecord_file
from code.cfa_utils.example_utils import feature_obj_detect

In [ ]:
print (tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Globals

In [ ]:
PROJECT_DIR = os.getcwd()

IMAGE_DIR = os.path.join(PROJECT_DIR, "data/jpeg_images")
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/annotations")

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564778509")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")
TFRECORD_DIR = os.path.join(PROJECT_DIR, "data/tfrecords")
TRAINING_SPLIT_TUPLE = (60,30,10)

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")
                                    
# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd"

## Data

In [ ]:
# this is the label_map as a dict
#    {'smallHotDrink': 2, 'nuggBox': 5, 'sandBag': 6, 'smallFry': 8, 
#     'largeFry': 9, 'cfaSauce': 7, 'mediumColdDrink': 3, 'sandBox': 4, 
#   'hand': 1, 'spicyBag': 11, 'polySauce': 10}

# you might have to replace some names due to inconsistencies in labeling and the map

! sed -i 's/smHotDrink/smallHotDrink/g' data/annotations/*.xml
! sed -i 's/medColdDrink/mediumColdDrink/g' data/annotations/*.xml

### WRITE tfrecords
this CFA script will:
- read the /annotations/*.xml files
- for each one that is verified (e.g. using labelImg)
  - get the image file
  - encode the label & image into a Feature (a dict that is defined for object detection - but not really a standard)
  - then create a tf.Example from the Feature
  - then add the example to the tfrecord file
  
#### Note
The tfrecords should be sharded for performance reasons.   Ideally one per processor thread which will yield the best performance.

good exampmle -  
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md

In [ ]:
# make tfrecords
voc_to_tfrecord_file(IMAGE_DIR, ANNOTATION_DIR, LABEL_MAP, TFRECORD_DIR, TRAINING_SPLIT_TUPLE)

### READ tfrecords


In [ ]:
! ls data/tfrecords/train -l
! ls data/tfrecords/val -l
! ls data/tfrecords/test -l

In [ ]:

# note this supports a list - very handy if you sharded the tfrecord files
# to make it interesting, just use train + val + test (until we actually shard the files)
tfrecord_file_list = ['data/tfrecords/train/train.tfrecord',
                     'data/tfrecords/val/val.tfrecord',
                     'data/tfrecords/test/test.tfrecord']

In [ ]:
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_list)
print ("raw dataset:", type(raw_dataset))
print ("            ", raw_dataset)

### Feature Decoder Function
First, you need a Feature, definition.    Feature is just a dict.   There is a commonly used standard in SSD models - as long as you use the same Feature definition on training, validation, inference - it doesn't matter, as long as  it is consistent.   We are using as much Google code as possible - which leverages the VOC PASCAL XML format (originally).  So we did NOT make this up.    

At this time, there was no easy utility in models/research/object_detection -- but I'm sure there will be eventually.   I pulled the definition out of TfExampleDecoder

In [ ]:
example_decoder = TfExampleDecoder()

# Understanding the Dataset

When you read the tfrecord file, you get a tensorflow TFRecordDataSetV1  
V1 = Tensorflow 1.x  
V2 = is coming  

It is a tensor with a string in it.   It has not be decoded into a Feature.  

The Example Decoder function - taken from the tensorflow/models/research/object_detection/ software, will read the string and decode it into a "standard - object detection" Feature (which is simply a dict.  and returns a tensor.  Thus, this function is intended to be used as a .map() function.

DON'T WRITE THIS YOURSELF - wherever possible, you want to use the tensorflow utilities

### Thus,
Below you see we use that function and map across the raw dataset - which returns a new dataset of decoded dict



In [ ]:
example_dataset = raw_dataset.map(example_decoder.decode)
print ("raw_dataset:", type(raw_dataset))
print ("            ", raw_dataset)
print ("\nexample_dataset:", type(example_dataset))
print ("                 ", example_dataset)

In [ ]:
for n, e in enumerate(example_dataset.take(4)):
    print (n,type(e)) 
    image_decoded = e['image']
    print (type(image_decoded))
    print (image_decoded)
    if n == 4:
        break

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

In [ ]:
feature0

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.
  print ("feature:", type(feature), feature)
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
type(serialized_example)

## Tensorflow Serving
using the saved_model_cli (see notebook: ?)  
We know the input signature of the method:  
- Method name is: tensorflow/serving/predict   

The given SavedModel SignatureDef contains the following input(s):  
  inputs['serialized_example'] tensor_info:  
      dtype: DT_STRING  
      shape: ()  
      name: tf_example:0  


In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

In [ ]:
output_dict = predictor.predict(serialized_example_json)

In [ ]:
prediction_dataset = raw_dataset.map(tf.train.Example.FromString)